In [31]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from glob import glob
import time
import random
import requests
import os
import glob
import re
import csv


class WOS_list_export_excel():

    def __init__(self, driver_path, download_path):
        self.url_to_crawl = 'https://www.webofscience.com/wos/woscc/basic-search'
        self.driver_path = driver_path
        self.keys_path = keys_path
        self.download_path = download_path
        self.keys = keys
        self.gg = gg

    def start(self):
        chrome_options = webdriver.ChromeOptions()
        prefs = {'download.default_directory': str(self.download_path)}
        chrome_options.add_experimental_option('prefs', prefs)
        self.browser = webdriver.Chrome(self.driver_path,
                                        chrome_options=chrome_options)
        self.browser.maximize_window()

    def get_into_page(self, url):
        self.browser.get(url)
        time.sleep(5)
        self.browser.find_element(By.LINK_TEXT, '進階檢索').click()
        time.sleep(5)
        self.browser.find_element(By.ID, "advancedSearchInputArea").clear()
        self.browser.find_element(By.ID,
                                  "advancedSearchInputArea").send_keys(keys)
        self.browser.find_element(
            By.CSS_SELECTOR,
            "body > app-wos > main > div > div > div.holder > div > div > div.held > app-input-route > app-search-home > div.advanced-search-form > div > app-input-route > app-search-advanced > app-advanced-search-form > form > div.upper-search-preview-holder > div.search-preview-left-holder > div.query-adv-search-row.search-preview-holder > div > button.mat-focus-indicator.search.mat-flat-button.mat-button-base.mat-primary.ng-star-inserted"
        ).click()
        time.sleep(5)

    def search_work(self):  #參數為搜尋的字元
        time.sleep(5)
        soup = BeautifulSoup(self.browser.page_source, 'html.parser')
        page_result = soup.find('span', {
            'class': 'brand-blue'
        }).text.replace(',', '')
        pattern = "[0-9]+"
        self.page_int = int(page_result)
        self.page_all = (int(self.page_int) // 1000) + 1
        page_last = int(self.page_int) % 1000
        #print(category)
        # 日期
        print('搜尋時間： ' + time.strftime("%y/%m/%d") + ' ' +
              time.strftime("%H:%M:%S"))
        print('共 ' + str(self.page_int) + ' 筆')
        print('需要匯出 ' + str(self.page_all) + ' 次')

    def download_file(self, start, end):
        self.browser.find_element(
            By.CSS_SELECTOR, ".mat-button-wrapper > .font-size-20").click()
        time.sleep(2.2)
        self.browser.find_element(By.ID, "exportToExcelButton").click()
        time.sleep(2.2)
        self.browser.find_element(
            By.XPATH, '//*[@id="radio3"]/label/span[1]/span[2]').click()
        time.sleep(2.2)
        self.browser.find_element(By.CSS_SELECTOR, ".radio-group-last").click()
        time.sleep(2.2)
        self.browser.find_element(By.XPATH, "/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/form/div/fieldset/mat-radio-group/div[3]/mat-form-field[1]/div/div[1]/div[3]/input").clear()
        self.browser.find_element(By.XPATH, "/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/form/div/fieldset/mat-radio-group/div[3]/mat-form-field[1]/div/div[1]/div[3]/input").send_keys(self.start)
        time.sleep(2.2)
        self.browser.find_element(By.XPATH, "/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/form/div/fieldset/mat-radio-group/div[3]/mat-form-field[2]/div/div[1]/div[3]/input").clear()
        self.browser.find_element(By.XPATH, "/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/form/div/fieldset/mat-radio-group/div[3]/mat-form-field[2]/div/div[1]/div[3]/input").send_keys(self.end)
        time.sleep(2.2)
        self.browser.find_element(
            By.XPATH,
            "/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/form/div/div[1]/wos-select/button"
        ).click()
        time.sleep(2.2)
        self.browser.find_element(
            By.XPATH, "//*[@id='global-select']/div/div[2]/div[3]").click()
        time.sleep(2.2)
        self.browser.find_element(
            By.XPATH,
            "/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/form/div/div[2]/button[1]"
        ).click()
        time.sleep(10)

    def roll_page(self):
        self.start = 1
        self.end = 1000
        roll = 1
        while roll <= self.page_all:
            self.download_file(self.start, self.end)
            roll = roll + 1
            self.start = 1 + self.end
            self.end = 1000 * roll

    def close_driver(self):
        self.browser.quit()
        print('成功爬取WOS- ' + Dataset)

    def step_setting(self):
        self.start()
        self.get_into_page(self.url_to_crawl)
        print('正在爬取WOS')
        self.search_work()
        self.roll_page()
        self.close_driver()


if __name__ == '__main__':
    driver_path = r'C:\Users\user\Desktop\py\Driver\chromedriver.exe'  #chromedriver位置
    keys_path = r'C:\Users\user\Desktop\py\指令整理.xlsx'  #指令
    download_path = r'C:\Users\user\Desktop\py\衍伸書目\WOS'  #檔案下載位置
    ALLkeys = pd.read_excel(keys_path)
    ALLkeys = ALLkeys[72:96]
    gg = 23
    keys = ALLkeys['Final'].iloc[gg - 1]
    Dataset = ALLkeys['Dataset'].iloc[gg - 1]
    go = WOS_list_export_excel(driver_path, download_path)
    go.step_setting()

# 1. 台灣社會變遷基本調查(TSCS)✔️
# 2. 華人家庭動態資料庫(PSFD)✔️
# 3. 台灣教育長期追蹤(TEPS)✔️
# 4. 特殊教育長期追蹤資料庫(SNELS)✔️
# 5. 家庭收支調查(SFIE)✔️
# 6. 台灣世界價值觀調查(TWVS)✔️
# 7. 台灣青少年成長歷程研究(TYP)✔️
# 8. 台灣選舉調查資料(TEDS)✔️
# 9. 「台灣教育長期追蹤資料庫」後續調查(TEPS-B)✔️
# 10. 人力資源調查(MPS)✔️
# 11. 人力運用調查(MUS)✔️
# 12. 婦女婚育與就業調查(WMFES)✔️
# 13. 個人家戶數位機會(落差)(數位落差)(DDS)✔️
# 14. 台灣貧窮兒少資料庫(TDCYP)✔️
# 15. 台灣傳播調查資料庫(TCS)✔️
# 16. International Social Survey Programme (ISSP)✔️
# 17. 幼兒發展調查資料庫建置計畫✔️
# 18. 國人旅遊狀況調查✔️
# 19. 來臺旅客消費及動向調查✔️
# 20. 台灣社會意向調查✔️
# 21. 台灣營養健康狀況變遷調查✔️
# 22. 我國大學生政治社會化的定群追蹤研究✔️
# 23. 台灣政府文官調查✔️
# 24. 青少年藥物濫用之起因✔️

C:\Temp\TEMP\ipykernel_18764\430338975.py:34: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.browser = webdriver.Chrome(self.driver_path,
C:\Temp\TEMP\ipykernel_18764\430338975.py:34: DeprecationWarning: use options instead of chrome_options
  self.browser = webdriver.Chrome(self.driver_path,


正在爬取WOS
搜尋時間： 22/11/22 15:37:35
共 64 筆
需要匯出 1 次
成功爬取WOS- 台灣政府文官調查
